# 02 - Data from the Web

## Deadline
Wednesday October 25, 2017 at 11:59PM

## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated (i.e., you don't want your colleagues to generate unnecessary Web traffic during the peer review)
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code.

## Background
In this homework we will extract interesting information from www.topuniversities.com and www.timeshighereducation.com, two platforms that maintain a global ranking of worldwide universities. This ranking is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need!
You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. For this task, Postman with the Interceptor extension can help you greatly. We recommend that you watch this [brief tutorial](https://www.youtube.com/watch?v=jBjXVrS8nXs&list=PLM-7VG-sgbtD8qBnGeQM5nvlpqB_ktaLZ&autoplay=1) to understand quickly how to use it.

## Assignment
1. Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the [details page](https://www.topuniversities.com/universities/ecole-polytechnique-fédérale-de-lausanne-epfl).
Store the resulting dataset in a pandas DataFrame and answer the following questions:
- Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
- Answer the previous question aggregating the data by (c) country and (d) region.

Plot your data using bar charts and describe briefly what you observed.

2. Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

3. Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

4. Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

5. Can you find the best university taking in consideration both rankings? Explain your approach.

Hints:
- Keep your Notebook clean and don't print the verbose output of the requests if this does not add useful information for the reader.
- In case of tie, use the order defined in the webpage.

In [660]:
# Import libraries
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import seaborn
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
%matplotlib inline

In [661]:
# Make a request
url_main = 'https://www.topuniversities.com'  # Found with postman
r = requests.get(url_main + '/sites/default/files/qs-rankings-data/357051.txt')
print('Response status code: {0}\n'.format(r.status_code))
page_body = r.text

# Serialize the json data with json library
rank_json = json.loads(page_body)

Response status code: 200



In [662]:
rank_df = pd.DataFrame()
rank_df = rank_df.from_dict(rank_json['data']).head(200)
rank_df.stars
rank_df.drop(['logo', 'stars', 'nid','cc', 'score'], axis=1, inplace=True)
rank_df.set_index('core_id', inplace=True)
rank_df = rank_df[['title', 'rank_display', 'country', 'region', 'url']]
rank_df.head()

,title,rank_display,country,region,url
core_id,,,,,
410,Massachusetts Institute of Technology (MIT),1,United States,North America,/universities/massachusetts-institute-technolo...
573,Stanford University,2,United States,North America,/universities/stanford-university
253,Harvard University,3,United States,North America,/universities/harvard-university
94,California Institute of Technology (Caltech),4,United States,North America,/universities/california-institute-technology-...
95,University of Cambridge,5,United Kingdom,Europe,/universities/university-cambridge


In [663]:
def my_find(html_attributes, new_df_column_name, rank_df):
    _tag = html_attributes['tag']
    _class = html_attributes['class']
    # _list is a temporary list that will store the values found and then be converted in the df to be returned.
    _list = []
    for url in rank_df.url:
        # for every url contained in rank_df['url'], perform the corresponding request:
        uni_url = requests.get(url_main + url)
        uni_body = uni_url.text
        soup = BeautifulSoup(uni_body, 'html.parser')
        # look for <tag=_tag, class=_class>
        soup1 = soup.find(_tag, class_=_class)
        # if such tag has been found, look then for <tag=_tag, class='number> where the value 
        # we're interested in is stored! otherwise append -99
        if soup1:
            soup2 = soup1.find(_tag, class_='number') 
            # if such tag has been found, append its value to the _list, otherwise append -99
            if soup2:
                _list.append({new_df_column_name: soup2.text})
            else:
                _list.append({new_df_column_name: -99})
        else:
            _list.append({new_df_column_name: -99})
    # convert _list to dataframe and return it
    return pd.DataFrame.from_dict(_list).replace({r'\n': ''},\
                                                 regex=True).replace({r',': ''},\
                                                                     regex=True).apply(pd.to_numeric).astype(int)

In [664]:
# defining HTML tag and class attributes that we want to find
tofind = [{'tag':'div', 'class': 'total faculty'}, 
          {'tag':'div', 'class': 'inter faculty'}, 
          {'tag':'div', 'class': 'total student'}, 
          {'tag':'div', 'class': 'total inter'}]

# creating DataFrame with the data found (NaN values = -99)
details_df = pd.concat([my_find(tofind[0], 'fac_memb_tot', rank_df),
                        my_find(tofind[1], 'fac_memb_int', rank_df),
                        my_find(tofind[2], 'nb_stud_tot', rank_df),
                        my_find(tofind[3], 'nb_stud_int', rank_df)], axis=1)

# concatenate the DataFrames into a unique one
details_df.set_index(rank_df.index, inplace=True)
QS_df = pd.concat([rank_df, details_df], axis=1)

# cleaning the unique DataFrame (deleting the = in rank_display)
QS_df.drop(['url'], axis=1, inplace=True)
QS_df.rank_display = QS_df.rank_display.replace({r'=': ''}, regex=True).apply(pd.to_numeric).astype(int)

# Creating the faculty_members_ratio and number_of_students ratio
QS_df['fac_memb_ratio'] = QS_df.fac_memb_tot / QS_df.nb_stud_tot
QS_df['int_stud_ratio'] = QS_df.nb_stud_int / QS_df.nb_stud_tot

# Deleting what's useless
del details_df, tofind, rank_df, r, page_body, rank_json

In [665]:
# resetting the index, dropping the 'core_id' column wich is useless, and renaming the columns
# in order to prepare themerge with the corresponding TimesHigherEducation dataframe
QS_df.reset_index(inplace = True)
QS_df.drop('core_id', axis=1, inplace = True)
QS_df.columns = ['QS_name', 'QS_rank', 'country', 'region', 'QS_fac_memb_tot', 'QS_fac_memb_int',
                'QS_nb_stud_tot', 'QS_nb_stud_int', 'QS_fac_memb_ratio', 'QS_int_stud_ratio']
QS_df.head()

,QS_name,QS_rank,country,region,QS_fac_memb_tot,QS_fac_memb_int,QS_nb_stud_tot,QS_nb_stud_int,QS_fac_memb_ratio,QS_int_stud_ratio
0,Massachusetts Institute of Technology (MIT),1,United States,North America,2982,1679,11067,3717,0.269450,0.335863
1,Stanford University,2,United States,North America,4285,2042,15878,3611,0.269870,0.227422
2,Harvard University,3,United States,North America,4350,1311,22429,5266,0.193945,0.234785
3,California Institute of Technology (Caltech),4,United States,North America,953,350,2255,647,0.422616,0.286918
4,University of Cambridge,5,United Kingdom,Europe,5490,2278,18770,6699,0.292488,0.356899


# TIMES HIGHER EDUCATION

In [666]:
# Making the request, beautifully soupping for extracting the dataframe
URL = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/'\
                +'world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'
r = requests.get(URL)
print('r = {r} // status_code = {status}'.format(r=r,status=r.status_code))
r.content
soupp = BeautifulSoup(r.content,'html.parser')
rank_json = json.loads(r.text)
THE_df = pd.DataFrame()
THE_df = THE_df.from_dict(rank_json['data']).head(200)
THE_df.head()

r = <Response [200]> // status_code = 200


,aliases,location,member_level,name,nid,rank,rank_order,record_type,scores_citations,scores_citations_rank,...,scores_research,scores_research_rank,scores_teaching,scores_teaching_rank,stats_female_male_ratio,stats_number_students,stats_pc_intl_students,stats_student_staff_ratio,subjects_offered,url
0,University of Oxford,United Kingdom,0,University of Oxford,468,1,10,master_account,99.1,15,...,99.5,1,86.7,5,46 : 54,"20,409",38%,11.2,"Archaeology,Art, Performing Arts & Design,Biol...",/world-university-rankings/university-oxford
1,University of Cambridge,United Kingdom,0,University of Cambridge,470,2,20,master_account,97.5,29,...,97.8,3,87.8,3,45 : 55,"18,389",35%,10.9,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/university-cambridge
2,California Institute of Technology caltech,United States,0,California Institute of Technology,128779,=3,30,private,99.5,10,...,97.5,4,90.3,1,31 : 69,"2,209",27%,6.5,"Architecture,Biological Sciences,Business & Ma...",/world-university-rankings/california-institut...
3,Stanford University,United States,11,Stanford University,467,=3,40,private,99.9,4,...,96.7,5,89.1,2,42 : 58,"15,845",22%,7.5,"Archaeology,Architecture,Art, Performing Arts ...",/world-university-rankings/stanford-university
4,Massachusetts Institute of Technology,United States,0,Massachusetts Institute of Technology,471,5,50,private,100.0,1,...,91.9,9,87.3,4,37 : 63,"11,177",34%,8.7,"Architecture,Art, Performing Arts & Design,Bio...",/world-university-rankings/massachusetts-insti...


In [667]:
# select columns of our interest and changing their name
THE_df = THE_df[['name', 'aliases', 'location', 'rank', 'stats_number_students', 'stats_student_staff_ratio', 'stats_pc_intl_students']]
THE_df.columns=['THE_name', 'THE_aliases', 'THE_location', 'THE_rank','THE_nb_stud_tot', 'THE_stats_student_staff_ratio','THE_int_stud_ratio']
THE_df.head()

,THE_name,THE_aliases,THE_location,THE_rank,THE_nb_stud_tot,THE_stats_student_staff_ratio,THE_int_stud_ratio
0,University of Oxford,University of Oxford,United Kingdom,1,"20,409",11.2,38%
1,University of Cambridge,University of Cambridge,United Kingdom,2,"18,389",10.9,35%
2,California Institute of Technology,California Institute of Technology caltech,United States,=3,"2,209",6.5,27%
3,Stanford University,Stanford University,United States,=3,"15,845",7.5,22%
4,Massachusetts Institute of Technology,Massachusetts Institute of Technology,United States,5,"11,177",8.7,34%


# still the usual, hard-to-understand cleaning of the dataset

In [668]:
# THE_int_stud_ratio
THE_df['THE_int_stud_ratio'] = THE_df['THE_int_stud_ratio'].str.replace('%', '').astype('double')/100

# THE_nb_stud_tot
THE_df['THE_nb_stud_tot'] = THE_df['THE_nb_stud_tot'].str.replace(',', '').astype('int')

# THE_fac_memb_int is a missing data, I add it just to state it clearly
THE_df['THE_fac_memb_int'] = -99

# THE_fac_memb_tot
THE_df['THE_stats_student_staff_ratio']=THE_df['THE_stats_student_staff_ratio'].astype(float)
THE_df['THE_fac_memb_tot'] = THE_df['THE_nb_stud_tot']/THE_df['THE_stats_student_staff_ratio']
THE_df['THE_fac_memb_tot'] = THE_df['THE_fac_memb_tot'].astype(int)

# THE_nb_stud_int
THE_df['THE_nb_stud_int'] = THE_df['THE_nb_stud_tot']*THE_df['THE_int_stud_ratio']
THE_df['THE_nb_stud_int'] = THE_df['THE_nb_stud_int'].astype(int)

# THE_fac_memb_ratio
THE_df['THE_fac_memb_ratio'] = 1/THE_df['THE_stats_student_staff_ratio']
THE_df=THE_df.drop('THE_stats_student_staff_ratio', axis=1)

# THE_rank
THE_df['THE_rank'] = THE_df['THE_rank'].astype(str)
THE_df['THE_rank'] = THE_df['THE_rank'].replace({r'=': ''}, regex=True).apply(pd.to_numeric).astype(int)

# let's now have a look at the clean THE_df

#### Note that the aliases is always more precise than the name

## Those where our dataset, from now we try to merge

In [669]:
QS_df['country'].value_counts().index

Index(['United States', 'United Kingdom', 'Germany', 'Netherlands',
       'Australia', 'Japan', 'China', 'Switzerland', 'South Korea', 'Canada',
       'Sweden', 'Hong Kong', 'France', 'Italy', 'Belgium', 'Denmark', 'Spain',
       'India', 'Austria', 'New Zealand', 'Norway', 'Mexico', 'Taiwan',
       'Singapore', 'Chile', 'Ireland', 'Brazil', 'Finland', 'Saudi Arabia',
       'Argentina', 'Russia', 'Israel', 'South Africa', 'Malaysia'],
      dtype='object')

In [670]:
QS_US = QS_df['country'][QS_df['country']=='United States'].value_counts()['United States']
THE_US = THE_df['THE_location'][THE_df['THE_location']=='United States'].value_counts()['United States']
abs(THE_US-QS_US)

15

In [671]:
# Set russian federation equal russia for comparing countries
THE_df.loc[THE_df['THE_location']=='Russian Federation','THE_location']='Russia'

# find differences in listed countries
tmp1 = QS_df['country'].value_counts().index
tmp2 = THE_df['THE_location'].value_counts().index
my_diff = list(set(tmp1).symmetric_difference(set(tmp2)))
my_diff

['Malaysia',
 'Argentina',
 'Chile',
 'Brazil',
 'India',
 'Luxembourg',
 'Israel',
 'Saudi Arabia',
 'Mexico']

In [672]:
list(THE_df)
out3 = pd.DataFrame()
#out3.columns.values = list(THE_df)
out_3 = THE_df.iloc[0:0,:] 
out_3

,THE_name,THE_aliases,THE_location,THE_rank,THE_nb_stud_tot,THE_int_stud_ratio,THE_fac_memb_int,THE_fac_memb_tot,THE_nb_stud_int,THE_fac_memb_ratio


In [1009]:
# return dataframes df1,df2 with only the rows where index in column1 exist also in column2 and vice versa
def exclude_differences(df1,df2,column1,column2):
    tmp1 = df1[column1].value_counts().index
    tmp2 = df2[column2].value_counts().index
    my_diff = list(set(tmp1).symmetric_difference(set(tmp2)))
    out_1 = df1.copy()
    out_2 = df2.copy()
    out_3 = out_1.iloc[0:0,:].copy()
    unmatched_df = pd.DataFrame()
    
    for loc in my_diff:
        out_31 = out_1[out_1[column1] == loc] # we wanna keep track of non meargeable elements
        out_32 = out_2[out_2[column2] == loc]
        out_3 = pd.concat([out_3, out_31])
        out_3 = pd.concat([out_3, out_32])
        #out_3.append(out_31)
        
        out_1 = out_1[out_1[column1] != loc] # now we get rid of these rows
        out_2 = out_2[out_2[column2] != loc]
        
        
        
    return out_1, out_2, out_3

In [1035]:
QS_step1, THE_step1, non_mergeable = exclude_differences(QS_df,THE_df,'country','THE_location')

print('QS: ',QS_step1.shape[0],'    THE: ',THE_step1.shape[0])
print(non_mergeable.shape[0])

QS_step1.sort_values('country',inplace=True)
THE_step1.sort_values('THE_location',inplace=True)

QS:  187     THE:  199
14


##### Note that non_meargeable contains only a subset of all the non-meargeables rows

In [1036]:
# just a cross check
tmp1 = QS_step1['country'].value_counts().sort_index().index
tmp2 = THE_step1['THE_location'].value_counts().sort_index().index
print(tmp1 == tmp2)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True]


I am gonna set QS ranking as reference and do the matching :)


# MY FUNCTIONS ----------------------------------------------------------------------------

## What influences fuzz?? 

äàö _ 

not éè'-()

In [1037]:
string_tmp = THE_tmp.loc[(THE_tmp["THE_location"] == 'France'), "THE_name"].values[0]
string_tmp2 = QS_tmp.loc[(QS_tmp["country"] == 'Italy'), "QS_name"].values[3]
accented_string = 'Málaga'
# accented_string is of type 'unicode'
import unidecode
unaccented_string = unidecode.unidecode(accented_string)
unaccented_string2 = unidecode.unidecode(string_tmp)
# unaccented_string contains 'Malaga'and is of type 'str'
print(unaccented_string,'\n',unaccented_string2,'\n',string_tmp2 )
print(unidecode.unidecode('äèàäéöû_èüÈ'))
print(unidecode.unidecode(accented_string))


Malaga 
 Ecole Normale Superieure de Lyon 
 Scuola Superiore Sant'Anna Pisa di Studi Universitari e di Perfezionamento
aeaaeou_euE
Malaga


In [1038]:
import collections
import string

table = collections.defaultdict(lambda: None)
table.update({
    ord('é'):'e',
    ord('ô'):'o',
    ord('ö'):'o',
    ord(' '):' ',
    ord('\''):' ',
    ord('-'): ' ',
    ord('\N{NO-BREAK SPACE}'): ' ',
    ord('\N{EN SPACE}'): ' ',
    ord('\N{EM SPACE}'): ' ',
    ord('\N{THREE-PER-EM SPACE}'): ' ',
    ord('\N{FOUR-PER-EM SPACE}'): ' ',
    ord('\N{SIX-PER-EM SPACE}'): ' ',
    ord('\N{FIGURE SPACE}'): ' ',
    ord('\N{PUNCTUATION SPACE}'): ' ',
    ord('\N{THIN SPACE}'): ' ',
    ord('\N{HAIR SPACE}'): ' ',
    ord('\N{ZERO WIDTH SPACE}'): ' ',
    ord('\N{NARROW NO-BREAK SPACE}'): ' ',
    ord('\N{MEDIUM MATHEMATICAL SPACE}'): ' ',
    ord('\N{IDEOGRAPHIC SPACE}'): ' ',
    ord('\N{IDEOGRAPHIC HALF FILL SPACE}'): ' ',
    ord('\N{ZERO WIDTH NO-BREAK SPACE}'): ' ',
    ord('\N{TAG SPACE}'): ' ',
    })
table.update(dict(zip(map(ord,string.ascii_uppercase), string.ascii_lowercase)))
table.update(dict(zip(map(ord,string.ascii_lowercase), string.ascii_lowercase)))
table.update(dict(zip(map(ord,string.digits), string.digits)))

print('123 fôé BAR҉'.translate(table,))
print('Paris-Sorbonne University – Paris 4'.translate(table,))
print(string_tmp2.translate(table,))
print('éöàäèüÒàèò'.translate(table,))


123 foe bar
paris sorbonne university  paris 4
scuola superiore sant anna pisa di studi universitari e di perfezionamento
eo


In [1039]:
def compare_missing(THE,QS):    
    missing_num_list_predicted = []
    missing_num_list_observed = []
    loc_list_num_QS = []
    loc_list_num_THE = []

    for i,loc in enumerate(QS['country'].value_counts().sort_index().index):

        loc_num_QS = QS['country'].value_counts().sort_index()[loc]
        loc_list_num_QS.append(loc_num_QS)

        loc_num_THE = THE['THE_location'].value_counts().sort_index()[loc]
        loc_list_num_THE.append(loc_num_THE)

        missing_num_list_predicted.append(abs(loc_num_QS - loc_num_THE))

        missing_num_list_observed.append(unique_list[i]['QS_name'].isnull().sum())
    # end of the for loop


    d = {'country': list(QS['country'].value_counts().sort_index().index),\
         'predicted': missing_num_list_predicted,\
         'observed' : missing_num_list_observed,\
         'QS' : loc_list_num_QS,\
         'THE' : loc_list_num_THE\
        }
    df_missing_comparison = pd.DataFrame(data=d)
    return df_missing_comparison

In [1040]:
#### martinos version not anymore hehe
def merging_by_name(THE,QS,prob_limit):
    # creating two lists with the names of the universities from the two datasets
    the = THE.copy()
    qs = QS.copy()
    THE_name = list(the.loc[:,'THE_name'])
    QS_name = list(qs.loc[:,'QS_name'])

    #initializing a new column of the THE_df with the corresponding QS name found by the matching function
    # just to control that everything went smoothly
    the['THE_corresponding QS name'] = 'unknown'
    the['prob'] = 'unknown'

    # MATCHING FUNCTION
    # finding the probable corresponding name in the QS dataframe for each university
    for i,THE_uni in enumerate(THE_name):
        QS_uni, prob=process.extractOne(THE_uni, QS_name, scorer=fuzz.token_sort_ratio)
        if prob>=prob_limit: #if prob<87, I observed that the algorithm matches diffeent universities!! 97 is a good limit
            the.loc[(the['THE_name']== THE_uni) , "THE_corresponding QS name"] = QS_uni
            the.loc[(the['THE_name']== THE_uni) , 'prob'] = prob
                     
    # MERGING        
    Unique_df=pd.merge(the,qs, left_on='THE_corresponding QS name', right_on='QS_name', how = 'outer')
    return Unique_df

In [1041]:
def create_unique_df(THE,QS,prob):
    unique_list = []
    for loc in QS['country'].value_counts().sort_index().index:
        the= THE.loc[THE['THE_location'] == loc , :].copy()
        qs  = QS.loc[QS['country'] == loc , :].copy()
        unique = merging_by_name(the,qs,prob)
        unique = unique.dropna(axis=0,how='any')
        unique_list.append(unique)
    unique_out = pd.concat(unique_list)
    return unique_out

In [1042]:
def drop_matched_uni(df,name, list_of_matched_uni):
    out = df.copy()
    for x in out[name]:
        if x in list_of_matched_uni:
            out.drop(out[out[name] == x].index[0], inplace=True)
    return out

In [1043]:
def merging_by_aliases(the,qs,prob_limit):
    # creating two lists with the names of the universities from the two datasets
    THE_aliases = list(the.loc[:,'THE_aliases'])
    QS_name = list(qs.loc[:,'QS_name'])

    #initializing a new column of the THE_df with the corresponding QS name found by the matching function
    # just to control that everything went smoothly
    the['THE_corresponding QS name'] = 'unknown'
    the['prob'] = 'unknown'

    # MATCHING FUNCTION
    # finding the probable corresponding name in the QS dataframe for each university
    for i,THE_uni in enumerate(THE_aliases):
        QS_uni, prob=process.extractOne(THE_uni, QS_name, scorer=fuzz.token_sort_ratio)
        if prob>=prob_limit: #if prob<87, I observed that the algorithm matches diffeent universities!! 97 is a good limit
            the.loc[(the['THE_aliases']== THE_uni) , "THE_corresponding QS name"] = QS_uni
            the.loc[(the['THE_aliases']== THE_uni) , 'prob'] = prob
                     
    # MERGING        
    Unique_df=pd.merge(the,qs, left_on='THE_corresponding QS name', right_on='QS_name', how = 'outer')
    return Unique_df

def create_unique_df_aliases(THE,QS,prob):
    unique_list = []
    for loc in QS['country'].value_counts().sort_index().index:
        the= THE.loc[THE['THE_location'] == loc , :].copy()
        qs  = QS.loc[QS['country'] == loc , :].copy()
        unique = merging_by_aliases(the,qs,prob)
        unique = unique.dropna(axis=0,how='any')
        unique_list.append(unique)
    unique_out = pd.concat(unique_list)
    return unique_out

In [1044]:
def remove_accent_in_name(df,name):
    out=df#.copy()
    for x in out[name]:
        x = unidecode.unidecode(x)
    return out

In [1045]:
def remove_characters_in_name(df,name):
    out=df.copy()
    for x in out[name]:
        x = x.translate(table,)
    return out


In [1046]:
def append_to_non_mergeable(non_mergeable,THE,QS,errors_THE,errors_QS):
    out = non_mergeable.copy()

    for x in errors_THE:
        out = pd.concat([out,THE.loc[THE['THE_name'] == x]])
    for x in errors_QS:
        out = pd.concat([out,QS.loc[QS['QS_name'] == x]])

    return out

## STEP 2 take out 100% fit

In [1047]:
Unique_df = create_unique_df(THE_step1,QS_step1,100)

In [1048]:
QS_matched_uni = list(Unique_df['QS_name'])
THE_matched_uni = list(Unique_df['THE_name'])
print(len(THE_matched_uni))

THE_step2 = drop_matched_uni(THE_step1,'THE_name',THE_matched_uni)
QS_step2 = drop_matched_uni(QS_step1,'QS_name', QS_matched_uni)

print(Unique_df.shape[0])
print(THE_step1.shape[0],THE_step2.shape[0])
print(QS_step1.shape[0],QS_step2.shape[0])

107
107
199 92
187 80


## STEP 3 go to 90%

In [1049]:
THE_step2_tmp = remove_accent_in_name(THE_step2,'THE_name') # doesn't work though
QS_step2_tmp = remove_accent_in_name(QS_step2,'QS_name')

In [1050]:
Unique_df2 = create_unique_df(THE_step2_tmp,QS_step2_tmp,90) #90 since 100 gave nothing

In [1051]:
THE_matched_uni = list(Unique_df2['THE_name'])
QS_matched_uni = list(Unique_df2['QS_name'])

THE_step3 = drop_matched_uni(THE_step2,'THE_name',THE_matched_uni)
QS_step3 = drop_matched_uni(QS_step2,'QS_name', QS_matched_uni)

print(Unique_df2.shape[0])
print(THE_step2.shape[0],THE_step3.shape[0])
print(QS_step2.shape[0],QS_step3.shape[0])

37
92 55
80 43


## STEP 4 go to 80%

In [1052]:
# do not re-run this cell if you don't wanna increase the non meargeable dataset! haha
AAA = 0

In [1053]:
THE_step3_tmp = remove_characters_in_name(THE_step3,'THE_name') # doesn't work though
QS_step3_tmp = remove_characters_in_name(QS_step3,'QS_name')

Unique_df3 = create_unique_df(THE_step3_tmp,QS_step3_tmp,80) 

In [1054]:
errors = ["Trinity College Dublin",'University College Dublin']
if AAA == 0 :
    non_mergeable = pd.concat([non_mergeable,THE_step3.loc[THE_step3['THE_name'] == "Trinity College Dublin"]]) 
    non_mergeable = pd.concat([non_mergeable,QS_step3.loc[QS_step3['QS_name'] == "University College Dublin"]]) 
    AAA = 1
Unique_df3 = Unique_df3.loc[Unique_df3['THE_name'] != "Trinity College Dublin"] #the only one with an error

errors
#Unique_df3.loc[Unique_df3['THE_name'] == "Trinity College Dublin",'THE_name'].values[0]

['Trinity College Dublin', 'University College Dublin']

In [1055]:
THE_matched_uni = list(Unique_df3['THE_name'])  + errors
QS_matched_uni = list(Unique_df3['QS_name']) + errors

THE_step4 = drop_matched_uni(THE_step3,'THE_name',THE_matched_uni)
QS_step4 = drop_matched_uni(QS_step3,'QS_name', QS_matched_uni)


non_mergeable = non_mergeable.drop_duplicates()
print(Unique_df3.shape[0])
print(non_mergeable.shape[0])
print(THE_step3.shape[0],THE_step4.shape[0])
print(QS_step3.shape[0],QS_step4.shape[0])

8
16
55 46
43 34


## STEP 5 go to 70%

In [1056]:
Unique_df4 = create_unique_df(THE_step4,QS_step4,70) 


errors = ['University of Sussex','University of Reading',\
          'University of East Anglia','University of Bath',\
          'University of Leicester','University of Dundee']

errors_THE = ['University of Sussex','University of East Anglia','University of Dundee','University of Leicester']
errors_QS = ['University of Reading','University of Bath']

Unique_df4_tmp = Unique_df4.copy()

for x in Unique_df4['QS_name']:
    if x in errors:
        Unique_df4 = Unique_df4[Unique_df4['QS_name'] != x]
for x in Unique_df4['THE_name']:
    if x in errors:
        Unique_df4 = Unique_df4[Unique_df4['THE_name'] != x]


non_mergeable = append_to_non_mergeable(non_mergeable,THE_step4,QS_step4,\
                                                    errors_THE,errors_QS)
non_mergeable = non_mergeable.drop_duplicates()
#non_mergeable = non_mergeable.drop_duplicates(subset='QS_name', keep="last")

errors = ['University of Sussex','University of Reading',\
          'University of East Anglia','University of Bath',\
          'University of Leicester','University of Dundee']

errors_THE = ['University of Sussex','University of East Anglia','University of Dundee','University of Leicester']
errors_QS = ['University of Reading','University of Bath']

Unique_df4_tmp = Unique_df4.copy()

for x in Unique_df4['QS_name']:
    if x in errors:
        Unique_df4 = Unique_df4[Unique_df4['QS_name'] != x]
for x in Unique_df4['THE_name']:
    if x in errors:
        Unique_df4 = Unique_df4[Unique_df4['THE_name'] != x]


non_mergeable = append_to_non_mergeable(non_mergeable,THE_step4,QS_step4,\
                                                    errors_THE,errors_QS)
non_mergeable = non_mergeable.drop_duplicates()
#non_mergeable = non_mergeable.drop_duplicates(subset='QS_name', keep="last")

THE_matched_uni = list(Unique_df4['THE_name'])  + errors_THE
QS_matched_uni = list(Unique_df4['QS_name']) + errors_QS

THE_step5 = drop_matched_uni(THE_step4,'THE_name',THE_matched_uni)
QS_step5 = drop_matched_uni(QS_step4,'QS_name', QS_matched_uni)


print(Unique_df4.shape[0])
print(non_mergeable.shape[0])
print(THE_step4.shape[0],THE_step5.shape[0])
print(QS_step4.shape[0],QS_step5.shape[0])

3
22
46 39
34 29


## Final STEP all.

In [1057]:
AAA = 0

Unique_df5 = create_unique_df(THE_step5,QS_step5,10) #90 since 100 gave nothing

rights = ['University of Tübingen','LMU Munich','University of Freiburg','Scuola Superiore Sant’Anna']
my_diff = list(set(rights).symmetric_difference(set(list(Unique_df5['THE_name']))))
Unique5 = Unique_df5.copy()
for x in my_diff:
    Unique_df5 = Unique_df5[Unique_df5['THE_name'] != x] 
#Unique_df5

THE_matched_uni = list(Unique_df5['THE_name']) 
QS_matched_uni = list(Unique_df5['QS_name']) 
THE_step6 = drop_matched_uni(THE_step5,'THE_name',THE_matched_uni)
QS_step6 = drop_matched_uni(QS_step5,'QS_name', QS_matched_uni)

if AAA == 0:
    non_mergeable = pd.concat([non_mergeable,THE_step6,QS_step6])

## check: 

In [1058]:
num_merged = pd.concat([Unique_df,Unique_df2,Unique_df3,Unique_df4,Unique_df5]).shape[0]

non_mergeable = non_mergeable.drop_duplicates()
num_non_merged = non_mergeable.shape[0]
print ('merged: ',num_merged,'  non-merged',num_non_merged,'\nmissing',(200-num_merged)*2-num_non_merged)

merged:  159   non-merged 82 
missing 0


In [1063]:
with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
    display(non_mergeable[['QS_name', 'THE_aliases']])

,QS_name,THE_aliases
113,Universiti Malaya (UM),NaN
74,Universidad de Buenos Aires (UBA),NaN
137,Pontificia Universidad Católica de Chile (UC),NaN
199,Universidad de Chile,NaN
120,Universidade de São Paulo,NaN
182,Universidade Estadual de Campinas (Unicamp),NaN
171,Indian Institute of Technology Delhi (IITD),NaN
178,Indian Institute of Technology Bombay (IITB),NaN
189,Indian Institute of Science (IISc) Bangalore,NaN
178,NaN,University of Luxembourg
